# Biology experiment 3 - Leech Action Potential
## - Plot Intracellular Recordings

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to Hide/Show the code."></form>''')

#### Run the cell below. Click on "Select Data File" button and select the text file of the recording data. Afer a while the plot will show up at the bottom. Use the same button if you want to select another file.
#### Use the slider below the figure to select a time range to display. You can also enter the range directly in the text box.
#### Click on "Low-pass Filter" button to filter out the oscillatory noise arised from the internal circuit of the amplifier. Use the slider on the right to set the cut-off frequency. Lower cut-off frequency will reduce noise better but also attenuate the amplitude of spikes since spikes also have high frequency component. 400 Hz cut-off frequency is recommended if you are observing spikes. 200-250 Hz cut-off frequency is recommended when you are measuring passive properties.

### Note: When you answer the questions using the recording data, remember to specify which piece of data you use and show your work in questions involving calculation.

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import Javascript,display
from tkinter import Tk, filedialog
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

root = Tk()
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

def select_files(*arg):
    global file_name
    file_name = filedialog.askopenfilename(initialdir=os.getcwd(),title="Select data file",filetypes=[("Text files","*.txt")])
    if file_name:
        display(Javascript('IPython.notebook.execute_cells([IPython.notebook.get_selected_index()+1])'))

SelectFileButton = widgets.Button(description='Select Data File',button_style='primary')
SelectFileButton.on_click(select_files)
display(SelectFileButton)

In [ ]:
headers = ['V','I']
parameters = pd.read_csv(open(file_name),names=headers,nrows=3)
dt = [float(s) for s in parameters.iloc[0,0].split() if s.replace('.','',1).isdigit()][0] # time step (ms)
dV = [float(s) for s in parameters.iloc[1,0].split() if s.replace('.','',1).isdigit()][0] # Voltage unit (mV)
dI = [float(s) for s in parameters.iloc[2,0].split() if s.replace('.','',1).isdigit()][0] # Current unit (nA)
Fs = 1000/dt
data = pd.read_csv(open(file_name),names=headers,skiprows=4)
N = data.shape[0]
t = dt*np.arange(N)
T = t[-1]
V = dV*data['V'].values
I = dI*data['I'].values

freq_cutoff = 400
bFilt,aFilt = signal.butter(2,freq_cutoff*2/Fs,btype='lowpass')
V_filt = signal.filtfilt(bFilt,aFilt,V)
I_filt = signal.filtfilt(bFilt,aFilt,I)
t_range = [0,T]

def plot_traces(filt,cutoff,t_range,fig):
    [x1,x2] = [max(np.floor(t_range[0]/T*N).astype(int),0) , min(np.ceil(t_range[1]/T*N).astype(int),N)+1]
    plt.close('all')
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    if filt:
        ax1.plot(t[x1:x2],V[x1:x2],'cyan')
        ax1.plot(t[x1:x2],V_filt[x1:x2],'blue')
        ax1.legend(['Raw V','Filtered V'],loc=1)
        ax2.plot(t[x1:x2],I[x1:x2],'pink')
        ax2.plot(t[x1:x2],I_filt[x1:x2],'red')
        ax2.legend(['Raw I','Filtered I'],loc=1)
    else:
        ax1.plot(t[x1:x2],V[x1:x2],'b')
        ax2.plot(t[x1:x2],I[x1:x2],'r')
    ax1.set_xlim(t_range)
    ax2.set_xlim(t_range)
    ax1.set_ylabel('mV')
    ax2.set_ylabel('nA')
    ax2.set_xlabel('Time (ms)')
    plt.show()

w_reset = widgets.Button(description='Reset',icon='history',button_style='primary')
w_fig = widgets.ToggleButton(value=False,description='Interactive plot',icon='window-restore',button_style='success')
w_filt = widgets.ToggleButton(value=False,description='Low-pass Filter',button_style='info',icon='play')
w_cutoff = widgets.FloatSlider(value=freq_cutoff,min=10,max=Fs/2,step=1,disabled=True,continuous_update=False)
w_t_range = widgets.FloatRangeSlider(value=t_range,min=0,max=T,step=1,continuous_update=False,readout_format='.0f',layout=Layout(width='60%'))

def reset_default(*args):
    w_filt.value = False
    w_cutoff.value = freq_cutoff
    w_t_range.value = t_range
w_reset.on_click(reset_default)

def interactive_fig(*arg):
    if w_fig.value:
        w_fig.icon = 'window-maximize'; w_fig.description = 'Inline plot'
        %matplotlib notebook
        %matplotlib notebook
    else:
        w_fig.icon = 'window-restore'; w_fig.description = 'Interactive plot'
        %matplotlib inline
w_fig.observe(interactive_fig,'value')

def switch_filt(*arg):
    if w_filt.value:
        w_cutoff.disabled = False
    else:
        w_cutoff.disabled = True
w_filt.observe(switch_filt,'value')

def set_cutoff(*arg):
    global V_filt,I_filt
    bFilt,aFilt = signal.butter(2,w_cutoff.value*2/Fs,btype='lowpass')
    V_filt = signal.filtfilt(bFilt,aFilt,V)
    I_filt = signal.filtfilt(bFilt,aFilt,I)
w_cutoff.observe(set_cutoff,'value')

out = interactive_output(plot_traces,{'filt':w_filt,'cutoff':w_cutoff,'t_range':w_t_range,'fig':w_fig})
ui = VBox([HBox([w_reset,w_fig]),out,HBox([Label('Display Time Range (ms):'),w_t_range]),HBox([w_filt,Label('Cut-off frequency'),w_cutoff])])
display(ui)